### Imports

In [1]:
import pandas as pd 
import os
import tqdm
import dask.dataframe as dd


#### Load IMDB Movie Information 

In [2]:
imdb_dir = "./Datasets/imdb-data-cleaned"


# Dictionary to store dataframes
imdb_data = {}

# List all files in the IMDB directory
for filename in os.listdir(imdb_dir):
    if filename.endswith('.csv'):  # IMDB files are typically tab-separated
        file_path = os.path.join(imdb_dir, filename)
        # Remove the .tsv extension to create the dictionary key
        key = filename.replace('.csv', '')
        # Read the TSV file
        imdb_data[key] = pd.read_csv(file_path, sep=',', low_memory=False)
        print(f"Loaded {filename} with shape {imdb_data[key].shape}")

Loaded title.basics.aka.csv with shape (135077, 9)
Loaded title.basics.csv with shape (83955, 9)
Loaded title.crew.csv with shape (83955, 3)
Loaded title.episode.csv with shape (46, 4)
Loaded title.principals.csv with shape (1523925, 6)
Loaded title.ratings.csv with shape (83955, 3)


### Create Dataframe with Movie Features 

In [3]:
# Start with the base features from title.basics
movie_features = imdb_data["title.basics"][["tconst", "primaryTitle", "isAdult", "startYear", "runtimeMinutes", "genres"]]

# Convert genres from string to list
movie_features['genres'] = movie_features['genres'].fillna('').str.split(',')

# Merge ratings data
movie_features = movie_features.merge(
    imdb_data["title.ratings"][["tconst", "averageRating", "numVotes"]], 
    on="tconst", 
    how="left"
)

# Merge crew data (directors and writers)
movie_features = movie_features.merge(
    imdb_data["title.crew"][["tconst", "directors", "writers"]], 
    on="tconst", 
    how="left"
)

# Convert directors and writers from string to list
movie_features['directors'] = movie_features['directors'].fillna('').str.split(',')
movie_features['writers'] = movie_features['writers'].fillna('').str.split(',')

# Handle principals data (cast and crew)
principals = imdb_data["title.principals"][["tconst", "nconst", "category"]]
# Group by movie and category to get lists of people in each role
principals_grouped = principals.groupby(['tconst', 'category'])['nconst'].agg(list).reset_index()
# Pivot to create columns for each category
principals_pivot = principals_grouped.pivot(
    index='tconst',
    columns='category',
    values='nconst'
).reset_index()

# Merge principals data with main features
movie_features = movie_features.merge(
    principals_pivot,
    on="tconst",
    how="left"
)

# Fill NaN values in list columns with empty lists
list_columns = ['genres', 'directors', 'writers'] + list(principals_pivot.columns[1:])
for col in list_columns:
    if col in movie_features.columns:
        movie_features[col] = movie_features[col].fillna('').apply(lambda x: [] if x == '' else x)

print("Final dataset shape:", movie_features.shape)
movie_features.head()

C:\Users\User\AppData\Local\Temp\ipykernel_16212\1723179424.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_features['genres'] = movie_features['genres'].fillna('').str.split(',')


Final dataset shape: (83955, 23)


,tconst,primaryTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes,directors,writers,...,archive_sound,casting_director,cinematographer,composer,director,editor,producer,production_designer,self,writer
0,tt0000001,Carmencita,0,1894,1,"[Documentary, Short]",5.7,2133,[nm0005690],[\N],...,[],[],[nm0374658],[],[nm0005690],[],[nm0005690],[],[nm1588970],[]
1,tt0000003,Poor Pierrot,0,1892,5,"[Animation, Comedy, Romance]",6.4,2167,[nm0721526],[\N],...,[],[],[],[nm1335271],[nm0721526],[nm5442200],"[nm1770680, nm0721526]",[],[],[]
2,tt0000007,Corbett and Courtney Before the Kinetograph,0,1894,1,"[Short, Sport]",5.3,900,"[nm0005690, nm0374658]",[\N],...,[],[],[nm0374658],[],"[nm0005690, nm0374658]",[],"[nm0005690, nm0249379]",[],[],[]
3,tt0000008,Edison Kinetoscopic Record of a Sneeze,0,1894,1,"[Documentary, Short]",5.4,2278,[nm0005690],[\N],...,[],[],[nm0374658],[],[nm0005690],[],[nm0005690],[],[],[]
4,tt0000010,Leaving the Factory,0,1895,1,"[Documentary, Short]",6.8,7865,[nm0525910],[\N],...,[],[],[nm0525910],[],[nm0525910],[],[nm0525910],[],[],[]


### Load User Information 

In [4]:
### Ok I want to add imdb ID to links.csv as a column
lens_links = pd.read_csv("./Datasets/ml-32m-cleaned/links.csv")
lens_ratings = pd.read_csv("./Datasets/ml-32m-cleaned/ratings.csv")


In [5]:
lens_data = lens_ratings.merge(
    lens_links[["imdbId", "movieId"]],
    on = "movieId",
    how = "inner"
              )

In [6]:
lens_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31972560 entries, 0 to 31972559
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
 4   imdbId     int64  
dtypes: float64(1), int64(4)
memory usage: 1.2 GB


In [7]:
### We do not need movieID anymore it is redundant. We just need IMDBId
### We won't be using timestamp
lens_data =lens_data.drop(columns = ["movieId", "timestamp"])


In [8]:
lens_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31972560 entries, 0 to 31972559
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   userId  int64  
 1   rating  float64
 2   imdbId  int64  
dtypes: float64(1), int64(2)
memory usage: 731.8 MB


In [11]:
movie_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83955 entries, 0 to 83954
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   tconst               83955 non-null  object 
 1   primaryTitle         83955 non-null  object 
 2   isAdult              83955 non-null  int64  
 3   startYear            83955 non-null  object 
 4   runtimeMinutes       83955 non-null  object 
 5   genres               83955 non-null  object 
 6   averageRating        83955 non-null  float64
 7   numVotes             83955 non-null  int64  
 8   directors            83955 non-null  object 
 9   writers              83955 non-null  object 
 10  actor                83955 non-null  object 
 11  actress              83955 non-null  object 
 12  archive_footage      83955 non-null  object 
 13  archive_sound        83955 non-null  object 
 14  casting_director     83955 non-null  object 
 15  cinematographer      83955 non-null 

In [9]:
print(lens_data.head(10))
movie_features.head(10)

   userId  rating  imdbId
0       1     4.0  114388
1       1     1.0  113627
2       1     2.0  112682
3       1     5.0  115012
4       1     5.0  114746
5       1     2.0  112431
6       1     1.0  112818
7       1     5.0  112445
8       1     3.0  112573
9       1     5.0   75314


,tconst,primaryTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes,directors,writers,...,archive_sound,casting_director,cinematographer,composer,director,editor,producer,production_designer,self,writer
0,tt0000001,Carmencita,0,1894,1,"[Documentary, Short]",5.7,2133,[nm0005690],[\N],...,[],[],[nm0374658],[],[nm0005690],[],[nm0005690],[],[nm1588970],[]
1,tt0000003,Poor Pierrot,0,1892,5,"[Animation, Comedy, Romance]",6.4,2167,[nm0721526],[\N],...,[],[],[],[nm1335271],[nm0721526],[nm5442200],"[nm1770680, nm0721526]",[],[],[]
2,tt0000007,Corbett and Courtney Before the Kinetograph,0,1894,1,"[Short, Sport]",5.3,900,"[nm0005690, nm0374658]",[\N],...,[],[],[nm0374658],[],"[nm0005690, nm0374658]",[],"[nm0005690, nm0249379]",[],[],[]
3,tt0000008,Edison Kinetoscopic Record of a Sneeze,0,1894,1,"[Documentary, Short]",5.4,2278,[nm0005690],[\N],...,[],[],[nm0374658],[],[nm0005690],[],[nm0005690],[],[],[]
4,tt0000010,Leaving the Factory,0,1895,1,"[Documentary, Short]",6.8,7865,[nm0525910],[\N],...,[],[],[nm0525910],[],[nm0525910],[],[nm0525910],[],[],[]
5,tt0000012,The Arrival of a Train,0,1896,1,"[Documentary, Short]",7.4,13361,"[nm0525908, nm0525910]",[\N],...,[],[],[nm0525910],[],"[nm0525908, nm0525910]",[],"[nm0525908, nm0525910]",[],"[nm2880396, nm9735580, nm0525900, nm9735581, n...",[]
6,tt0000013,The Photographical Congress Arrives in Lyon,0,1895,1,"[Documentary, Short]",5.7,2047,[nm0525910],[\N],...,[],[],[nm0525910],[],[nm0525910],[],[nm0525910],[],"[nm0525908, nm1715062]",[]
7,tt0000014,The Waterer Watered,0,1895,1,"[Comedy, Short]",7.1,6094,[nm0525910],[\N],...,[],[],[nm0525910],[],[nm0525910],[],[nm0525910],[],[],[]
8,tt0000015,Around a Cabin,0,1894,2,"[Animation, Short]",6.1,1258,[nm0721526],[\N],...,[],[],[],[],[nm0721526],[],[],[],[],[]
9,tt0000016,Boat Leaving the Port,0,1895,1,"[Documentary, Short]",5.9,1645,[nm0525910],[\N],...,[],[],[nm0525910],[],[nm0525910],[],[nm0525910],[],"[nm0525900, nm9735581]",[]


##### Create User InformationTable

In [31]:
import pandas as pd
import numpy as np
from tqdm import tqdm

def create_enhanced_user_profiles(ratings_df, movie_features):
    """
    Create user profiles by rating buckets, with summary statistics from `movie_features`.
    Includes:
      - Basic numeric stats (year/runtime/rating/etc.)
      - Proportion of genres
      - Top directors/writers lists
    """
    # --------------------------------------------------------------------------
    # 1. Copy dataframes to avoid modifying the originals
    # --------------------------------------------------------------------------
    movie_features = movie_features.copy()
    ratings_df = ratings_df.copy()
    
    # --------------------------------------------------------------------------
    # 2. Preprocess movie_features: remove 'tt' prefix, convert numeric columns
    # --------------------------------------------------------------------------
    movie_features['numeric_tconst'] = (
        movie_features['tconst']
        .str.replace('tt', '', regex=False)
        .astype(float)
    )

    movie_features['startYear'] = pd.to_numeric(movie_features['startYear'], errors='coerce')
    movie_features['runtimeMinutes'] = pd.to_numeric(movie_features['runtimeMinutes'], errors='coerce')

    # --------------------------------------------------------------------------
    # 3. Helper function to gather bucket-level stats
    # --------------------------------------------------------------------------
    def get_bucket_stats(movies_df):
        """Given a subset of movie_features, return relevant statistics."""
        if len(movies_df) == 0:
            return {}
        
        stats = {}
        # A. Basic movie IDs and count
        stats['movie_ids'] = list(movies_df['tconst'])
        stats['count'] = len(movies_df)
        
        # B. Numeric features
        numeric_features = {
            'adult_prop': 'isAdult',
            'year_avg': 'startYear',
            'year_var': 'startYear',
            'runtime_avg': 'runtimeMinutes',
            'runtime_var': 'runtimeMinutes',
            'rating_avg': 'averageRating',
            'rating_var': 'averageRating',
            'numvotes_avg': 'numVotes',
            'numvotes_var': 'numVotes'
        }
        
        for stat_name, column in numeric_features.items():
            if column not in movies_df.columns:
                continue
            if 'avg' in stat_name:
                stats[stat_name] = movies_df[column].mean(skipna=True)
            elif 'var' in stat_name:
                stats[stat_name] = movies_df[column].var(skipna=True)
            elif 'prop' in stat_name:
                # For 'adult_prop', store the fraction that are adult
                # (assuming isAdult is 0 or 1).
                stats[stat_name] = movies_df[column].mean(skipna=True)
        
        # C. Genre proportions
        #    Assuming 'genres' is already a list of genres for each movie
        all_genres = []
        for genre_list in movies_df['genres']:
            if isinstance(genre_list, list):
                # Filter out "\N" or empty strings
                valid_genres = [g.strip() for g in genre_list if g and g != '\\N']
                all_genres.extend(valid_genres)
        
        if all_genres:
            genre_counts = pd.Series(all_genres).value_counts()
            genre_proportions = genre_counts / genre_counts.sum()
            # Store as a dict: {genre: proportion}
            stats['genre_proportions'] = dict(genre_proportions)
        
        # D. Directors and Writers
        #    We'll store top 3 for each, but you can expand as needed.
        
        # Directors
        all_directors = []
        for directors_list in movies_df['directors']:
            if isinstance(directors_list, list):
                valid_directors = [d.strip() for d in directors_list if d and d != '\\N']
                all_directors.extend(valid_directors)
        
        if all_directors:
            director_counts = pd.Series(all_directors).value_counts()
            stats['top_directors'] = list(director_counts.head(3).index)
        
        # Writers
        all_writers = []
        for writers_list in movies_df['writers']:
            if isinstance(writers_list, list):
                valid_writers = [w.strip() for w in writers_list if w and w != '\\N']
                all_writers.extend(valid_writers)
        
        if all_writers:
            writer_counts = pd.Series(all_writers).value_counts()
            stats['top_writers'] = list(writer_counts.head(3).index)
        
        return stats

    # --------------------------------------------------------------------------
    # 4. Main user-profile creation
    # --------------------------------------------------------------------------
    user_profiles = []
    
    # Define your rating buckets
    buckets = [(0,1), (2, 3), (4 , 5)] #Ratings are between 0 and 5
    #buckets = [(0, 2), (3, 4), (5, 6), (7, 8), (9, 10)]
    
    unique_users = ratings_df['userId'].unique()
    pbar = tqdm(unique_users, desc="Creating user profiles")

    for user_id in pbar:
        user_ratings = ratings_df[ratings_df['userId'] == user_id]
        total_ratings = len(user_ratings)
        
        # Basic info about the user's rating behavior
        profile = {
            'userId': user_id,
            'total_ratings': total_ratings,
            'rating_avg': user_ratings['rating'].mean(),
            'rating_std': user_ratings['rating'].std(),
            'rating_proportions': dict(user_ratings['rating'].value_counts(normalize=True))
        }
        
        # Store references for bucket stats
        bucket_counts = {}
        bucket_stats_dict = {}
        
        # Process each rating bucket
        for low, high in buckets:
            bucket_name = f'bucket_{low}_{high}'
            bucket_ratings = user_ratings[
                (user_ratings['rating'] >= low) & (user_ratings['rating'] <= high)
            ]
            
            # Find the movies in this bucket
            bucket_movies = movie_features[
                movie_features['numeric_tconst'].isin(bucket_ratings['imdbId'])
            ]
            
            # Gather stats
            bucket_stats = get_bucket_stats(bucket_movies)
            bucket_counts[bucket_name] = bucket_stats.get('count', 0)
            bucket_stats_dict[bucket_name] = bucket_stats
            
            # Add each stat to the user profile with a prefix
            for stat_name, stat_value in bucket_stats.items():
                profile[f'{bucket_name}_{stat_name}'] = stat_value
        
        # ----------------------------------------------------------------------
        # 5. Interaction features (ratios, differences, etc.) between buckets
        # ----------------------------------------------------------------------
        for i, (low1, high1) in enumerate(buckets):
            for j, (low2, high2) in enumerate(buckets[i+1:], i+1):
                bucket1_name = f'bucket_{low1}_{high1}'
                bucket2_name = f'bucket_{low2}_{high2}'

                # Ratio of counts
                ratio_name = f'ratio_{bucket1_name}_to_{bucket2_name}'
                denom = float(bucket_counts[bucket2_name])
                profile[ratio_name] = float((
                    float(bucket_counts[bucket1_name]) / denom if denom > 0 else 0
                ))

                # Differences of average stats (year/runtime/rating)
                for stat in ['year_avg', 'runtime_avg', 'rating_avg']:
                    if (bucket_stats_dict[bucket1_name].get(stat) is not None and
                        bucket_stats_dict[bucket2_name].get(stat) is not None):
                        diff_name = f'{stat}_diff_{bucket1_name}_vs_{bucket2_name}'
                        profile[diff_name] = (
                            bucket_stats_dict[bucket1_name][stat] -
                            bucket_stats_dict[bucket2_name][stat]
                        )
        
        # Another example: ratio of high-ratings to low-ratings
        high_count = float(bucket_counts.get('bucket_7_8', 0) + bucket_counts.get('bucket_9_10', 0))
        low_count = float((
            bucket_counts.get('bucket_0_2', 0) +
            bucket_counts.get('bucket_3_4', 0) +
            1e-6  # avoid zero-division
        ))
        profile['high_vs_low_ratio'] = high_count / low_count
        
        # ----------------------------------------------------------------------
        # 6. Overall stats for all movies the user rated
        # ----------------------------------------------------------------------
        all_movies = movie_features[
            movie_features['numeric_tconst'].isin(user_ratings['imdbId'])
        ]
        overall_stats = get_bucket_stats(all_movies)
        for k, v in overall_stats.items():
            profile[f'overall_{k}'] = v
        
        user_profiles.append(profile)

    pbar.close()
    
    # Convert list of dicts into a DataFrame
    return pd.DataFrame(user_profiles)

# ---------------------------------------------------------------------
# Example Usage in a Notebook (after you define `combined` and `movie_features`)
# ---------------------------------------------------------------------
enhanced_user_profiles2 = create_enhanced_user_profiles(combined, movie_features)
display(enhanced_user_profiles2.head())


Creating user profiles: 100%|██████████| 200948/200948 [1:57:41<00:00, 28.46it/s]  


,userId,total_ratings,rating_avg,rating_std,rating_proportions,bucket_0_2_movie_ids,bucket_0_2_count,bucket_0_2_adult_prop,bucket_0_2_year_avg,bucket_0_2_year_var,...,overall_year_var,overall_runtime_avg,overall_runtime_var,overall_rating_avg,overall_rating_var,overall_numvotes_avg,overall_numvotes_var,overall_genre_proportions,overall_top_directors,overall_top_writers
0,1,141,3.531915,1.537870,"{5.0: 0.40425531914893614, 4.0: 0.184397163120...","[tt0034583, tt0045793, tt0056172, tt0059113, t...",39.0,0.0,1988.153846,171.923077,...,221.714083,122.609929,725.596758,7.678014,0.397442,316521.531915,1.811400e+11,"{'Drama': 0.3125, 'Comedy': 0.1477272727272727...","[nm0001661, nm0000217, nm0000416]","[nm0001410, nm0000416, nm0000184]"
1,2,52,4.269231,1.122242,"{5.0: 0.5961538461538461, 4.0: 0.2115384615384...","[tt0109040, tt0109686, tt0110148, tt0110912]",4.0,0.0,1994.000000,0.000000,...,73.923077,110.673077,391.792986,6.942308,0.772293,344699.653846,3.543127e+11,"{'Drama': 0.2396694214876033, 'Comedy': 0.1900...","[nm0001060, nm0873779, nm0936374]","[nm0560329, nm0363125, nm0322368]"
2,3,147,3.588435,1.014789,"{4.0: 0.43537414965986393, 3.0: 0.183673469387...","[tt0054033, tt0060782, tt0088161, tt0102216, t...",12.0,0.0,1988.666667,162.787879,...,117.339111,119.258503,591.987513,7.182313,0.706534,405335.149660,2.439600e+11,"{'Adventure': 0.1649746192893401, 'Action': 0....","[nm0000709, nm0000229, nm0000165]","[nm0363125, nm0560329, nm0000184]"
3,4,27,2.629630,1.043225,"{2.0: 0.4074074074074074, 3.0: 0.2592592592592...","[tt0097162, tt0099052, tt0120032, tt0120657, t...",14.0,0.0,1997.428571,11.648352,...,60.028490,109.000000,372.461538,6.744444,0.381795,170791.259259,4.959259e+10,"{'Action': 0.14285714285714285, 'Horror': 0.11...","[nm0001532, nm0742341, nm0769874]","[nm0000184, nm0371249, nm0636002]"
4,5,33,3.272727,0.910794,"{3.0: 0.48484848484848486, 4.0: 0.333333333333...","[tt0109686, tt0110148, tt0110912, tt0111161]",4.0,0.0,1994.000000,0.000000,...,2.354167,124.636364,539.238636,7.466667,0.799792,638410.060606,5.806468e+11,"{'Drama': 0.19540229885057472, 'Action': 0.172...","[nm0000318, nm0000126, nm0873779]","[nm0761498, nm0560329, nm0363125]"


In [32]:
enhanced_user_profiles2.to_pickle('data.pkl')

In [33]:
enhanced_user_profiles2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200948 entries, 0 to 200947
Data columns (total 81 columns):
 #   Column                                     Non-Null Count   Dtype  
---  ------                                     --------------   -----  
 0   userId                                     200948 non-null  int64  
 1   total_ratings                              200948 non-null  int64  
 2   rating_avg                                 200948 non-null  float64
 3   rating_std                                 200948 non-null  float64
 4   rating_proportions                         200948 non-null  object 
 5   bucket_0_2_movie_ids                       172578 non-null  object 
 6   bucket_0_2_count                           172578 non-null  float64
 7   bucket_0_2_adult_prop                      172578 non-null  float64
 8   bucket_0_2_year_avg                        172578 non-null  float64
 9   bucket_0_2_year_var                        152071 non-null  float64
 10  bucket_0

In [20]:
print(enhanced_user_profiles2.head(1))

   userId  total_ratings  rating_avg  rating_std  \
0       1            141    3.531915     1.53787   

                                  rating_proportions  \
0  {5.0: 0.40425531914893614, 4.0: 0.184397163120...   

                                bucket_0_2_movie_ids  bucket_0_2_count  \
0  [tt0034583, tt0045793, tt0056172, tt0059113, t...              39.0   

   bucket_0_2_adult_prop  bucket_0_2_year_avg  bucket_0_2_year_var  ...  \
0                    0.0          1988.153846           171.923077  ...   

   overall_year_var  overall_runtime_avg  overall_runtime_var  \
0        221.714083           122.609929           725.596758   

   overall_rating_avg  overall_rating_var  overall_numvotes_avg  \
0            7.678014            0.397442         316521.531915   

  overall_numvotes_var                          overall_genre_proportions  \
0         1.811400e+11  {'Drama': 0.3125, 'Comedy': 0.1477272727272727...   

               overall_top_directors                overall_t

In [10]:
enhanced_user_profiles = pd.read_pickle('data.pkl')

In [14]:
enhanced_user_profiles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200948 entries, 0 to 200947
Data columns (total 81 columns):
 #   Column                                     Non-Null Count   Dtype  
---  ------                                     --------------   -----  
 0   userId                                     200948 non-null  int64  
 1   total_ratings                              200948 non-null  int64  
 2   rating_avg                                 200948 non-null  float64
 3   rating_std                                 200948 non-null  float64
 4   rating_proportions                         200948 non-null  object 
 5   bucket_0_2_movie_ids                       172578 non-null  object 
 6   bucket_0_2_count                           172578 non-null  float64
 7   bucket_0_2_adult_prop                      172578 non-null  float64
 8   bucket_0_2_year_avg                        172578 non-null  float64
 9   bucket_0_2_year_var                        152071 non-null  float64
 10  bucket_0

In [39]:
movie_features["tconst"]

0        tt0000001
1        tt0000003
2        tt0000007
3        tt0000008
4        tt0000010
           ...    
83950    tt9911196
83951    tt9914192
83952    tt9914644
83953    tt9916270
83954    tt9916362
Name: tconst, Length: 83955, dtype: object

In [11]:
import dask.dataframe as dd
import pandas as pd
import os

def create_combined_features_table(ratings_df, user_profiles_df, movie_features_df, output_folder="combined_chunks"):
    print("Starting to create combined features table...")

    # Convert imdbId and tconst to numeric for merging
    ratings_df["imdbId"] = pd.to_numeric(ratings_df["imdbId"], errors='coerce')
    
    # Remove "tt" prefix if it exists in movie_features_df["tconst"]
    movie_features_df["tconst"] = movie_features_df["tconst"].astype(str).str.replace("tt", "", regex=False)
    movie_features_df["tconst"] = pd.to_numeric(movie_features_df["tconst"], errors='coerce')

    # Reduce memory usage
    user_profiles_df["userId"] = user_profiles_df["userId"].astype("int32")
    ratings_df["userId"] = ratings_df["userId"].astype("int32")

    # Convert to Dask DataFrame
    ratings_ddf = dd.from_pandas(ratings_df, npartitions=10)
    user_profiles_ddf = dd.from_pandas(user_profiles_df, npartitions=10)
    movie_features_ddf = dd.from_pandas(movie_features_df, npartitions=10)

    # Merge in a streaming manner
    print("Merging ratings with user profiles...")
    combined_ddf = ratings_ddf.merge(user_profiles_ddf, on="userId", how="left")

    print("Merging with movie features...")
    combined_ddf = combined_ddf.merge(movie_features_ddf, left_on="imdbId", right_on="tconst", how="left")

    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Save each partition as a separate Pickle file
    print(f"Saving merged data to {output_folder} in chunks...")
    for i, partition in enumerate(combined_ddf.to_delayed()):
        partition_df = partition.compute()  # Compute only one small partition at a time
        chunk_path = os.path.join(output_folder, f"chunk_{i}.pkl")
        partition_df.to_pickle(chunk_path)
        print(f"✅ Saved chunk {i} to {chunk_path}")

    print("\n✅ Merging completed successfully! Data saved in chunks.")
    return output_folder  # Returning the folder path

# Run the function and save Pickle in chunks
output_folder = create_combined_features_table(
    ratings_df=lens_data,
    user_profiles_df=enhanced_user_profiles,
    movie_features_df=movie_features,
    output_folder="./Datasets/combined_chunks"
)



Starting to create combined features table...
Merging ratings with user profiles...
Merging with movie features...
Saving merged data to ./Datasets/combined_chunks in chunks...


ArrowMemoryError: malloc of size 3461226496 failed

In [ ]:
# 🔄 Load the saved Pickle files and concatenate when needed
import glob

def load_combined_features(output_folder="./Datasets/combined_chunks"):
    """Loads all saved Pickle files and combines them into a single DataFrame."""
    pickle_files = glob.glob(os.path.join(output_folder, "*.pkl"))
    dfs = [pd.read_pickle(f) for f in pickle_files]
    combined_df = pd.concat(dfs, ignore_index=True)
    return combined_df

# Load and inspect the final DataFrame
combined_features = load_combined_features(output_folder)

print("\nSample of combined features table:")
print(combined_features.head())

print("\nColumn information:")
print(combined_features.info())


In [41]:
def create_combined_features_table(ratings_df, user_profiles_df, movie_features_df, output_file):
    print("Starting to create combined features table...")

    # Convert imdbId and tconst to int to avoid merge issues and remove prefix
    ratings_df["imdbId"] = ratings_df["imdbId"].to_numeric(ratings_df["imdbId"], errors='coerce')
    if 'tt' in str(movie_features_df["tconst"].iloc[0]):
        movie_features_df["tconst"] = movie_features_df["tconst"].str.replace("tt", "")
    movie_features_df["tconst"] = pd.to_numeric(movie_features_df["tconst"], errors='coerce')

    # Reduce memory usage by converting data types
    user_profiles_df["userId"] = user_profiles_df["userId"].astype("int32")
    ratings_df["userId"] = ratings_df["userId"].astype("int32")
    # Convert to Dask DataFrames
    ratings_ddf = dd.from_pandas(ratings_df, npartitions=10)
    user_profiles_ddf = dd.from_pandas(user_profiles_df, npartitions=10)
    movie_features_ddf = dd.from_pandas(movie_features_df, npartitions=10)

    # Merge in chunks using Dask
    print("Merging ratings with user profiles...")
    combined_ddf = ratings_ddf.merge(user_profiles_ddf, on="userId", how="left")

    print("Merging with movie features...")
    combined_ddf = combined_ddf.merge(movie_features_ddf, left_on="imdbId", right_on="tconst", how="left")

    # Compute the final DataFrame
    #combined_df = combined_ddf.compute()  # Converts Dask DataFrame back to Pandas
    combined_ddf.to_pickle(output_file)
    print(f"Combined features table saved to: {output_file}")

    return combined_ddf

# Run the function with Dask optimizations
combined_features = create_combined_features_table(
    ratings_df=lens_data,
    user_profiles_df=enhanced_user_profiles,
    movie_features_df=movie_features,
    output_file="./Datasets/features.pkl"
)

# Display a sample of the combined table
print("\nSample of combined features table:")
print(combined_features.head())

# Display column info
print("\nColumn information:")
print(combined_features.info())


Starting to create combined features table...
Merging ratings with user profiles...
Merging with movie features...


ArrowMemoryError: malloc of size 3370626048 failed

In [16]:
# Save the combined features table to CSV
output_path = "./Datasets/combined_features.pkl"
combined_features.to_pickle(output_path, index=False)
print(f"Combined features table saved to: {output_path}")

Combined features table saved to: ./Datasets/combined_features.csv


### Remove Redundant Information. MovieID IMDBId
### Focus on director and writer remove the rest
### For Missing Values Add Averages.
### Convert Genres Into Binary Features

In [13]:

### Divide into 5 buckets 
### Movies rated from [0,2] [3,4][5,6][7,8][9,10]
### For each bucket 
### Movie ID's of movies in the bucket
### Adult movie proportion
### startYear Average
### StartYear Variance
### Runtime Average
### Runtime Variance
### Genres Proportions
### List of Directors for movies
### List of writers for movies
### Average Numvotes
### Variance Numvotes


### User Information
    ### Average Rating 
    ### Rating Deviation   

    



In [14]:

### Both Liked and Watched
    ### Adult Proportions
    ### StartYear Average 
    ### StartYear Variance
    ### Runtime Average
    ### Runtime Variance
    ### Genres Proportions
    ### Directors  
    ### Writers
    ### Average Rating 
    ### Variance Rating 
    ### Average NumVotes
    ### Variance Numvotes 

### User Information
    ### Average Rating 
    ### Rating Deviation   
    ### Proportion of Rating

    



## Normalize Data

#### Load Dataframe

In [4]:
import os
import pandas as pd
# Get current working directory
print("Current working directory:", os.getcwd())

# Use os.path.join for path construction
file_path = os.path.join(os.getcwd(), "Datasets", "enhanced_user_profiles2.csv")
df = pd.read_csv(file_path)

Current working directory: c:\Users\User\Desktop\personalprojects\RecSystem


#### Normalization 

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200948 entries, 0 to 200947
Data columns (total 81 columns):
 #   Column                                     Non-Null Count   Dtype  
---  ------                                     --------------   -----  
 0   userId                                     200948 non-null  int64  
 1   total_ratings                              200948 non-null  int64  
 2   rating_avg                                 200948 non-null  float64
 3   rating_std                                 200948 non-null  float64
 4   rating_proportions                         200948 non-null  object 
 5   bucket_0_2_movie_ids                       172578 non-null  object 
 6   bucket_0_2_count                           172578 non-null  float64
 7   bucket_0_2_adult_prop                      172578 non-null  float64
 8   bucket_0_2_year_avg                        172578 non-null  float64
 9   bucket_0_2_year_var                        152071 non-null  float64
 10  bucket_0

In [6]:
os.getcwd()

'c:\\Users\\User\\Desktop\\personalprojects\\RecSystem'